# Modell laden + LoRA Fine-Tuning auf Lichess-Spiele


In [1]:
%pip install -U transformers torch requests huggingface_hub peft datasets accelerate


  Using cached transformers-5.2.0-py3-none-any.whl.metadata (32 kB)
Using cached transformers-5.2.0-py3-none-any.whl (10.4 MB)
   ---------------------------------------- 0.0/557.0 kB ? eta -:--:--
   ---------------------------------------- 557.0/557.0 kB 13.1 MB/s  0:00:00

  Attempting uninstall: datasets

    Found existing installation: datasets 4.5.0

    Uninstalling datasets-4.5.0:

      Successfully uninstalled datasets-4.5.0

   ---------------------------------------- 0/3 [datasets]
   ---------------------------------------- 0/3 [datasets]
   ---------------------------------------- 0/3 [datasets]
   ---------------------------------------- 0/3 [datasets]
   ---------------------------------------- 0/3 [datasets]
   ---------------------------------------- 0/3 [datasets]
   ---------------------------------------- 0/3 [datasets]
   ---------------------------------------- 0/3 [datasets]
   ---------------------------------------- 0/3 [datasets]
   -------------------------

In [8]:
from huggingface_hub import login, whoami
import os

token = os.getenv("HF_TOKEN")
if token:
    login(token=token, add_to_git_credential=False)
print(whoami())


{'type': 'user', 'id': '699f22d299fe22837907a458', 'name': 'DavidKrst', 'fullname': 'David Krstevski', 'isPro': False, 'avatarUrl': '/avatars/660b589436ccd68475b7c83e3aedcebf.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'bachelorarbeit', 'role': 'fineGrained', 'createdAt': '2026-02-25T16:32:58.634Z', 'fineGrained': {'canReadGatedRepos': True, 'global': [], 'scoped': [{'entity': {'_id': '699f22d299fe22837907a458', 'type': 'user', 'name': 'DavidKrst'}, 'permissions': []}]}}}}


In [9]:
import json
import random
import re
import requests
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, default_data_collator
from peft import LoraConfig, get_peft_model, TaskType

model_id = "daavidhauser/chess-bot-3000-250m"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)
baseline_model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
baseline_model.eval()

print("Model geladen:", model_id, "| device:", device)


Loading weights:   0%|          | 0/146 [00:00<?, ?it/s]

Model geladen: daavidhauser/chess-bot-3000-250m | device: cpu


In [14]:
def load_lichess_games_uci(username: str, max_games: int = 200):
    url = f"https://lichess.org/api/games/user/{username}"
    headers = {"Accept": "application/x-ndjson"}
    params = {
        "max": max_games,
        "moves": "true",
        "pgnInJson": "false",
        "opening": "false",
        "clocks": "false",
        "evals": "false"
    }

    r = requests.get(url, headers=headers, params=params, timeout=30)
    r.raise_for_status()

    raw_games = [json.loads(line) for line in r.text.splitlines() if line.strip()]
    games = []
    for g in raw_games:
        games.append({
            "id": g.get("id"),
            "white": g.get("players", {}).get("white", {}).get("user", {}).get("name"),
            "black": g.get("players", {}).get("black", {}).get("user", {}).get("name"),
            "moves_uci": g.get("moves", "").split(),
        })
    return games


In [15]:
import chess

username = "TSMFTXH"  # Zielspieler
games = load_lichess_games_uci(username=username, max_games=200)
print(f"Geladene Spiele von {username}: {len(games)}")

examples = []
user_lower = username.lower()

for g in games:
    white = (g.get("white") or "").lower()
    black = (g.get("black") or "").lower()
    san_moves = g.get("moves_uci", [])  # Lichess liefert hier SAN-Tokens

    if white == user_lower:
        user_color = chess.WHITE
    elif black == user_lower:
        user_color = chess.BLACK
    else:
        continue

    board = chess.Board()
    uci_history = []

    for san in san_moves:
        try:
            move = board.parse_san(san)
        except Exception:
            break

        if board.turn == user_color and len(uci_history) > 0:
            context = " ".join(uci_history)
            target = move.uci()
            examples.append({"context": context, "target": target})

        uci_history.append(move.uci())
        board.push(move)

print("Trainingsbeispiele:", len(examples))
print("Beispiel:", examples[0] if examples else None)


Geladene Spiele von TSMFTXH: 200
Trainingsbeispiele: 8264
Beispiel: {'context': 'f2f4', 'target': 'g7g6'}


In [16]:
random.seed(42)
random.shuffle(examples)
split_idx = int(0.8 * len(examples))
train_examples = examples[:split_idx]
test_examples = examples[split_idx:]

print("Train:", len(train_examples), "| Test:", len(test_examples))


Train: 6611 | Test: 1653


In [17]:
UCI_RE = re.compile(r"\b([a-h][1-8][a-h][1-8][qrbn]?)\b")

def extract_first_uci(text: str):
    if not text:
        return None
    text = text.lower().replace("\n", " ")
    m = UCI_RE.search(text)
    return m.group(1) if m else None

def top1_pred_move(model, tokenizer, context: str, device: str):
    inputs = tokenizer(context, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=12,
            do_sample=False,
            num_beams=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    gen_ids = out[0][inputs["input_ids"].shape[1]:]
    gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    pred = extract_first_uci(gen_text)
    return pred, gen_text

def eval_top1_debug(model, tokenizer, test_data, device: str, debug_n: int = 20):
    if not test_data:
        return {"top1": 0.0, "total": 0, "correct": 0, "none_pred": 0, "debug_rows": []}

    correct = 0
    total = 0
    none_pred = 0
    debug_rows = []

    for ex in test_data:
        pred, raw = top1_pred_move(model, tokenizer, ex["context"], device)
        target = ex["target"]
        is_correct = (pred == target)

        if pred is None:
            none_pred += 1
        if is_correct:
            correct += 1
        total += 1

        if len(debug_rows) < debug_n:
            debug_rows.append({
                "target": target,
                "pred": pred,
                "raw_gen": raw,
                "correct": is_correct,
                "context_tail": ex["context"][-120:],
            })

    return {
        "top1": (correct / total) if total else 0.0,
        "total": total,
        "correct": correct,
        "none_pred": none_pred,
        "debug_rows": debug_rows,
    }

baseline_report = eval_top1_debug(baseline_model, tokenizer, test_examples, device, debug_n=20)
baseline_top1 = baseline_report["top1"]

print(f"Baseline Top-1: {baseline_top1:.4f}")
print(f"Correct/Total: {baseline_report['correct']}/{baseline_report['total']}")
print(f"None-Predictions: {baseline_report['none_pred']}")
print("\nDebug-Samples (target | pred | raw):")
for row in baseline_report["debug_rows"][:10]:
    print(f"{row['target']} | {row['pred']} | {row['raw_gen']!r}")


Baseline Top-1: 0.4949
Correct/Total: 818/1653
None-Predictions: 0

Debug-Samples (target | pred | raw):
d4f5 | d4f5 | 'd4f5 a1f1 f3e2 f1g1 f5h6 g1g2 e2f3 g2g1 h6f5 g1a1 h5h6 a1a8'
e1e2 | e1e2 | 'e1e2 e8e7 e2e1 a7a8 e1e2 a8a7 e2e1 a7a8 e1e2 a8a7 e2e1 a7a8'
e5c3 | e5c3 | 'e5c3 f3g2 c3d2 g2f3 d2d3 f3g2 d3b5 g4h4 b5h5 h4f4 h5g5 f4g5'
f1c1 | b3d4 | 'b3d4 f5d7 f1c1 c8c1 b2c1 g6f6 c1c2 h7h6 g2e4 d5e4 c2e4 d7c6'
c1b2 | c1b2 | 'c1b2 g8f6 f1b5 c8d7 e1g1 a7a6 b5c6 d7c6 f3e5 d8c7 f2f4 e7e6'
a5a7 | e3e1 | 'e3e1 b8b2 a5a7 f7f8 a7a8 f8f7 a8a7 f7f8 a7a8 f8f7 a8a7'
c7c6 | e8d6 | 'e8d6 c1f4 c8f5 f4d6 f5d3 d6f8 d3b1 a1b1 d8f8 d1f3 c7c6 e1e2'
b1d2 | b1d2 | 'b1d2 e7e5 e2e4 d5d4 c2c3 c6c5 c3d4 c5d4 d2c4 d8c7 d1b3 a7a6'
d2d3 | e1g1 | 'e1g1 e8g8 c2c4 c7c6 b1c3 d5c4 f3e5 c8e6 a2a4 b8d7 c1f4 d7e5'
f1e1 | e4g2 | 'e4g2 h3g2 g1g2 d8d5 g2g1 b7b5 c4a3 a7a6 a3c2 c6e5 c2e1 c5c4'


In [24]:
def tokenize_for_lm(example, tokenizer, max_length=256):
    context = example["context"]
    target = example["target"]
    full_text = f"{context} {target}"

    # Fix: feste Laenge fuer alle Batch-Sequenzen
    full = tokenizer(full_text, truncation=True, padding="max_length", max_length=max_length)
    ctx = tokenizer(context, truncation=True, padding="max_length", max_length=max_length)

    input_ids = full["input_ids"]
    labels = input_ids.copy()

    # Kontext maskieren, nur Target-Zug als Lernsignal
    mask_len = min(len(ctx["input_ids"]), len(labels))
    for i in range(mask_len):
        labels[i] = -100

    return {
        "input_ids": input_ids,
        "attention_mask": full["attention_mask"],
        "labels": labels,
    }

train_ds = Dataset.from_list(train_examples)
test_ds = Dataset.from_list(test_examples)

train_tok = train_ds.map(lambda x: tokenize_for_lm(x, tokenizer), remove_columns=train_ds.column_names)
test_tok = test_ds.map(lambda x: tokenize_for_lm(x, tokenizer), remove_columns=test_ds.column_names)

print(train_tok[0])


Map:   0%|          | 0/6611 [00:00<?, ? examples/s]

Map:   0%|          | 0/1653 [00:00<?, ? examples/s]

{'input_ids': [2428, 1601, 599, 1114, 3016, 3954, 3503, 3451, 2897, 1536, 76, 426, 888, 2190, 2357, 4018, 4191, 2876, 1839, 2778, 1382, 1013, 3072, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [25]:
lora_base = AutoModelForCausalLM.from_pretrained(model_id).to(device)

lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules="all-linear",
)

lora_model = get_peft_model(lora_base, lora_cfg)
lora_model.print_trainable_parameters()


Loading weights:   0%|          | 0/146 [00:00<?, ?it/s]

trainable params: 5,636,096 || all params: 253,739,008 || trainable%: 2.2212


In [ ]:
training_args = TrainingArguments(
    output_dir="outputs/lora-player",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=25,
    report_to=[],
    fp16=torch.cuda.is_available(),
    remove_unused_columns=False,
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    data_collator=default_data_collator,
)

trainer.train()


Epoch,Training Loss,Validation Loss


In [ ]:
lora_model.eval()

finetuned_report = eval_top1_debug(lora_model, tokenizer, test_examples, device, debug_n=20)
finetuned_top1 = finetuned_report["top1"]

print(f"Baseline Top-1:   {baseline_top1:.4f} ({baseline_report['correct']}/{baseline_report['total']})")
print(f"Fine-tuned Top-1: {finetuned_top1:.4f} ({finetuned_report['correct']}/{finetuned_report['total']})")
print(f"Delta:            {finetuned_top1 - baseline_top1:+.4f}")
print(f"Fine-tuned None-Predictions: {finetuned_report['none_pred']}")
print("\nFine-tuned Debug-Samples (target | pred | raw):")
for row in finetuned_report["debug_rows"][:10]:
    print(f"{row['target']} | {row['pred']} | {row['raw_gen']!r}")
